# Initialization

In [1]:
using CSV, DataFrames, JuMP, HiGHS, LinearAlgebra, DataStructures, Statistics, GLPK
baltic_demand = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/Demand_Baltic.csv", delim='\t'))
ports = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/ports.csv", delim='\t'))
distances = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/dist_dense.csv", delim='\t'))
baltic_fleet = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_Baltic.csv", delim='\t'))
fleet_data = DataFrame(CSV.File("/Users/augusthogsted/WD/Bachelor projekt/LINERLIB-master/data/fleet_data.csv", delim='\t'))
;

In [2]:
commodities = []
for i in 1:size(baltic_demand, 1)
    push!(commodities, string(baltic_demand.Origin[i], "s", "|", baltic_demand.Destination[i], "t"))
end

### Services:

The following array contains manual generated services to which the column generation will be applied.

In [3]:
# services = [["Feeder_800" "DEBRV" "NOKRS" "DEBRV"],
#             ["Feeder_800" "RUKGD" "FIKTK" "RUKGD"],
#             ["Feeder_450" "DEBRV" "NOSVG" "DEBRV"],
#             ["Feeder_450" "NOAES" "NOBGO" "NOAES"],
#             ["Feeder_450" "NOKRS" "SEGOT" "DKAAR" "NOKRS"],
#             ["Feeder_800" "DEBRV" "SEGOT" "DKAAR" "DEBRV"],
#             ["Feeder_450" "DEBRV" "NOSVG" "NOBGO" "NOAES" "DEBRV"],
#             ["Feeder_450" "SEGOT" "DKAAR" "PLGDY" "RUKGD" "SEGOT"],
#             ["Feeder_450" "DEBRV" "NOAES" "DEBRV" "RULED" "DEBRV"],
#             ["Feeder_450" "DEBRV" "RUKGD" "PLGDY" "DKAAR" "SEGOT" "DEBRV"],
#             ["Feeder_800" "PLGDY" "RUKGD" "RULED" "FIKTK" "FIRAU" "PLGDY"],
#             ["Feeder_450" "PLGDY" "RUKGD" "FIKTK" "RULED" "FIRAU" "PLGDY"],
#             ["Feeder_450" "NOKRS" "SEGOT" "DKAAR" "PLGDY" "RUKGD" "PLGDY" "DKAAR" "SEGOT" "NOKRS"],
#             ["Feeder_800" "NOAES" "NOBGO" "NOSVG" "NOKRS" "SEGOT" "DKAAR" "SEGOT" "NOKRS" "NOSVG" "NOBGO" "NOAES"]]

# services = [[33.0 2 "Feeder_800" "DEBRV" "NOKRS" "DEBRV"],
#                 [33.0 2 "Feeder_800" "NOKRS" "NOSVG" "NOBGO" "NOAES" "NOKRS"],
#                 [33.0 2 "Feeder_450" "NOKRS" "SEGOT" "DKAAR" "NOKRS"],
#                 [33.0 2 "Feeder_800" "DKAAR" "PLGDY" "RUKGD" "DKAAR"],
#                 [33.0 2 "Feeder_450" "SEGOT" "FIRAU" "FIKTK" "RULED" "SEGOT"]]

# services = [[33.0 2 "Feeder" "DEBRV" "DKAAR" "SEGOT" "FIRAU" "DEBRV"],
#             [33.0 2 "Feeder" "NOKRS" "DKAAR" "SEGOT" "NOBGO" "NOKRS"]]

services = [
    [200000 1 "Feeder_450" "DEBRV" "NOSVG" "NOBGO" "NOAES" "DEBRV"],
    [130000 1 "Feeder_450" "DEBRV" "RUKGD" "FIKTK" "FIRAU" "PLGDY" "DEBRV"],
    [97000 1 "Feeder_450" "DEBRV" "DKAAR" "SEGOT" "NOKRS" "DEBRV"],
    [100000 1 "Feeder_800" "DEBRV" "SEGOT" "RULED" "DEBRV"]
]


# Initialize an empty Set to store unique ports
service_ports = []
# Iterate through the subarrays and add the unique ports to the set
for service in services
    for port in service[4:end]
        push!(service_ports, port)
    end
end
service_ports = unique(service_ports);


In [4]:
# List of ports that have intersecting service
intersect_ports = []
for i in 1:length(services)
    for j in i+1:length(services)
        push!(intersect_ports, intersect([[row[4:end]...] for row in services][i], [[row[4:end]...] for row in services][j]))
    end
end
intersect_ports = collect(Set(i for j in intersect_ports for i in j)); # Collecting unique transshipment nodes

# Adding transshipment nodes where services intersect
services_w_transshipment = []
transshipment_nodes = []
i = 1
for service in services
    updated_service = []
    for port in service
        if port in intersect_ports
            push!(updated_service, port * string(i)) # vi behøves i realiteten ikke at update vores serivces
            push!(transshipment_nodes, port * string(i))
        else
            push!(updated_service, port)
        end
    end
    push!(services_w_transshipment, updated_service)
    i += 1
end
transshipment_nodes = unique(transshipment_nodes);

In [5]:
# Source and terminal adjacency list
s_t_adj = []
for port in service_ports
    for node in transshipment_nodes
        if port == node[1:5]
            push!(s_t_adj, [string(port, "s"), node, parse.(Float64, ports[ports.UNLocode .== port, 9][1])  + ports[ports.UNLocode .== port, 12][1]])
            push!(s_t_adj, [node, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1]) + ports[ports.UNLocode .== port, 12][1]])
        end
    end
    if port ∉ intersect_ports
        push!(s_t_adj, [string(port, "s"), port, parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
        push!(s_t_adj, [port, string(port, "t"), parse.(Float64, ports[ports.UNLocode .== port, 9][1])])
    end
end

# Transshipment nodes adjacency list
transshipment_adj = []
for node in intersect_ports, i in transshipment_nodes, j in transshipment_nodes
    if j!=i && contains(i, node) && contains(j, node)
        push!(transshipment_adj, [i, j, parse.(Float64, ports[ports.UNLocode .== node, 10][1])]) # Skal vi også have move omkostninger med, da vi skal load og disch?
    end
end

# Service ports adjacency list
ports_adj = []
test = []
for service in 1:length(services_w_transshipment)
    for port in 5:length(services_w_transshipment[service]) # Hvad gør vi med port call omkostninger?
        push!(ports_adj, [services_w_transshipment[service][port-1], services_w_transshipment[service][port], 0.0]) # ports[ports.UNLocode .== services_w_transshipment[service][port][1:5], 12][1]]
    end
end

# Penalty edges adjacency list
penalty_adj = []
for k in 1:size(baltic_demand,1)
    push!(penalty_adj, [string(baltic_demand.Origin[k], "s"), string(baltic_demand.Destination[k], "t"), 1000])
end

dummy_adj = []
for port in 1:length(service_ports)
    push!(dummy_adj, [string(service_ports[port], "t"), nothing, 0.0])
end

edges = vcat(ports_adj, s_t_adj, transshipment_adj, penalty_adj, dummy_adj);

In [6]:
function get_G(edges)
    # Create an empty adjacency list
    graph = Dict{String, Vector{Tuple{Any, Float64}}}()  # Use Tuple to store (target, VC)
    #graph = Dict{String, Vector{Tuple{String, Float64}}}()  # Use Tuple to store (target, VC)

    # Populate the adjacency list
    for edge in edges
        source, target, weight = edge
        if haskey(graph, source)
            push!(graph[source], (target, weight))
        else
            graph[source] = [(target, weight)]
        end
    end

    return graph
end

get_G (generic function with 1 method)

In [7]:
function dijkstra(G, source, destination)
    dist = Dict{Any, Float64}()
    prev = Dict{Any, Any}()
    Q = Set(keys(G))

    for v in Q
        dist[v] = Inf
        prev[v] = nothing
    end

    dist[source] = 0.0

    while !isempty(Q)
        min_value = Inf
        min_key = string()
        for i in Q
            if dist[i] <= min_value
                min_value = dist[i]
                min_key = i
            end
        end
        u = min_key

        if dist[u] == Inf
            break
        end
        if u == destination
            break
        end
        
        delete!(Q, u)

        for v in G[u]
            if haskey(G, v[1])
                alt = dist[u] + v[2]
                if alt < dist[v[1]]
                    dist[v[1]] = alt
                    prev[v[1]] = u
                end
            else
                break
            end
        end
    end

    S = []
    u = destination
    push!(S, u)

    while prev[u] != nothing
        push!(S, u)
        u = prev[u]
    end
    push!(S, source)
    
    return reverse(S[2:end]), dist[destination]
end


dijkstra (generic function with 1 method)

## Flowproblem

### Initialization

In [21]:
current_paths = []
current_costs = []
current_commodities = []
;

## Basis

In [22]:
# Bases med penalty edges
for i in 1:length(penalty_adj)
    push!(current_paths, penalty_adj[i][1:2])
    push!(current_costs, penalty_adj[i][3])
    append!(current_commodities, i)
end


# Edges with capacity constraint
i_j = []
for edge in edges
    if edge ∈ (ports_adj)
        push!(i_j, string(edge[1], "|", edge[2]))
    end
end

n_ij = size(i_j, 1) # Number of edges
ij = 1:n_ij # Range of edges

# Initializing parameters
n_k = size(baltic_demand.Origin, 1) # Number of commodities
K = 1:n_k

n_s = size(services_w_transshipment, 1) # Number of services
S = 1:n_s

n_ff = size(penalty_adj, 1) # Number of penalty edges
ff = 1:n_ff
index_ff = 1:n_k # Index of penalty paths

n_p = size(current_paths, 1) # Number of paths
P = 1:n_p

n_mv = size(baltic_fleet, 1)
v = 1:n_mv
;

In [23]:
# Initial base including penalty edges
PK = Matrix{Float64}(undef, n_k, 0) # Type of commodity for each path
mvs = Matrix{Float64}(undef, n_mv, 0) # Type of vessel on each commodity
aij = Matrix{Float64}(undef, n_ij, 0)
uij = Matrix{Float64}(undef, n_ij, 0) # vessel capacity if service s uses edge (i,j), else 0

# Constraint matrices for commodidty paths PK and edges used
for i in P
    ak = zeros(Float64, n_ij)
    pk = zeros(Float64, n_k)
    for j in 2:length(current_paths[i])
        if string(current_paths[i][j-1], "|", current_paths[i][j]) ∈ i_j
            ak[findall(x -> x == string(current_paths[i][j-1], "|", current_paths[i][j]), i_j)[1]] = 1
        end
    end
    pk[findall(x -> x == string(current_paths[i][1], "|", current_paths[i][end]), commodities)[1]] = 1
    aij = [aij ak]
    PK = [PK pk]
end

# Constraint matrix for edge capacities
for i in 1:length(services_w_transshipment)
    us = zeros(Float64, n_ij)
    for j in 5:length(services_w_transshipment[i])
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
    end
    uij = [uij us]
end

# Constraint matrix for service vessels
for i in services
    mv = zeros(Float64, n_mv)
    mv[findall(x -> x == i[3], baltic_fleet."Vessel class")[1]] = 1
    mvs = [mvs mv]
end

P = 1:length(current_paths)
A = 1:size(aij, 2)

1:22

### Basis problem with penalty only

In [24]:
flowModel = Model(GLPK.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, 0 <= y[S] <= 1)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum(current_costs[j]*x[j] for j in P))
@constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j] for j in A) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
@constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= baltic_fleet.Quantity[i])
#print(flowModel)
optimize!(flowModel)
solution_summary(flowModel)


* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 4.90400e+06
  Objective bound    : -Inf
  Dual objective value : 4.90400e+06

* Work counters
  Solve time (sec)   : 1.11103e-04


In [27]:
dual_Y_k = [dual(flowModel[:Y_k][i]) for i in K] # Dual variables for commodities at iteration 1 
dual_Y_ij = [dual(flowModel[:Y_ij][i]) for i in ij] # Dual variables for edges at iteration 1 
#reduced_cost = [-1.0] # Kickstarting while loop
;

16×0 Matrix{Float64}

In [28]:
# while any(reduced_cost .< 0)

append!(dual_Y_ij, zeros(Float64, length(edges) - length(ports_adj)))

# Initializing feasible paths
candidate_paths = []
candidate_costs = []
candidate_commodities = []

# Setting up new network with reduced cost
edges_w_dual = deepcopy(edges)
for i in 1:length(edges_w_dual)
    edges_w_dual[i][3] = edges_w_dual[i][3] - dual_Y_ij[i]
end

for i in 1:size(baltic_demand, 1)
    path, cost = dijkstra(get_G(edges_w_dual), string(baltic_demand.Origin[i],"s"), string(baltic_demand.Destination[i],"t"))
    push!(candidate_paths, path)
    push!(candidate_costs, cost)
    append!(candidate_commodities, i)
end


reduced_cost = candidate_costs - dual_Y_k

cp_r_k = candidate_paths[reduced_cost .< 0] #candidate_paths[reduced_cost .== minimum(reduced_cost)]

for i in 1:length(cp_r_k)
    ak = zeros(Float64, n_ij)
    pk = zeros(Float64, n_k)
    for j in 2:length(cp_r_k[i])
        if string(cp_r_k[i][j-1], "|", cp_r_k[i][j]) ∈ i_j
            ak[findall(x -> x == string(cp_r_k[i][j-1], "|", cp_r_k[i][j]), i_j)[1]] = 1
        end
    end
    pk[findall(x -> x == string(cp_r_k[i][1], "|", cp_r_k[i][end]), commodities)[1]] = 1
    aij = [aij ak]
    PK = [PK pk]
    push!(current_paths, cp_r_k[i])
    push!(current_costs, candidate_costs[reduced_cost .< 0][i])
    append!(current_commodities, findall(x -> x == string(cp_r_k[i][1], "|", cp_r_k[i][end]), commodities)[1])
end


P = 1:length(current_paths)
A = 1:size(aij, 2)

flowModel = Model(GLPK.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, 0 <= y[S] <= 1)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum(current_costs[j]*x[j] for j in P))
@constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j+n_ff] for j in A) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
@constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= baltic_fleet.Quantity[i])
optimize!(flowModel)
#print(flowModel)

dual_Y_k = [dual(flowModel[:Y_k][i]) for i in K]
dual_Y_ij = [dual(flowModel[:Y_ij][i]) for i in ij]



# # # #end
# # # primal_x = value.(x[P])
# # # primal_y = value.(y[S])


[-591.0, -358.0, -472.0, -554.0, -103.0, -703.0, -527.0, -422.0, -554.0, -591.0, -646.0, -422.0, -650.0, -103.0, -703.0, -527.0, -472.0, -650.0, -358.0, -517.0, -646.0, -517.0]


16-element Vector{Float64}:
 -341.4444444444446
    0.0
    0.0
 -102.99999999999994
 -553.9999999999998
    0.0
    0.0
    0.0
 -591.0000000000002
 -358.0
    0.0
    0.0
    0.0
 -507.0
   -9.999999999999964
 -517.0

In [29]:
print(flowModel)

Min 200000 y[1] + 130000 y[2] + 97000 y[3] + 100000 y[4] + 1000 x[1] + 1000 x[2] + 1000 x[3] + 1000 x[4] + 1000 x[5] + 1000 x[6] + 1000 x[7] + 1000 x[8] + 1000 x[9] + 1000 x[10] + 1000 x[11] + 1000 x[12] + 1000 x[13] + 1000 x[14] + 1000 x[15] + 1000 x[16] + 1000 x[17] + 1000 x[18] + 1000 x[19] + 1000 x[20] + 1000 x[21] + 1000 x[22] + 409 x[23] + 642 x[24] + 528 x[25] + 446 x[26] + 897 x[27] + 297 x[28] + 473 x[29] + 578 x[30] + 446 x[31] + 409 x[32] + 354 x[33] + 578 x[34] + 350 x[35] + 897 x[36] + 297 x[37] + 473 x[38] + 528 x[39] + 350 x[40] + 642 x[41] + 483 x[42] + 354 x[43] + 483 x[44]
Subject to
 Y_k[1] : x[1] + x[23] = 77
 Y_k[2] : x[2] + x[24] = 456
 Y_k[3] : x[3] + x[25] = 65
 Y_k[4] : x[4] + x[26] = 7
 Y_k[5] : x[5] + x[27] = 10
 Y_k[6] : x[6] + x[28] = 98
 Y_k[7] : x[7] + x[29] = 660
 Y_k[8] : x[8] + x[30] = 17
 Y_k[9] : x[9] + x[31] = 268
 Y_k[10] : x[10] + x[32] = 18
 Y_k[11] : x[11] + x[33] = 16
 Y_k[12] : x[12] + x[34] = 37
 Y_k[13] : x[13] + x[35] = 187
 Y_k[14] : x[14]

In [225]:
i_j[2:3]

2-element Vector{Any}:
 "NOSVG|NOBGO"
 "NOBGO|NOAES"

In [213]:
candidate_paths[reduced_cost .< 0]

7-element Vector{Any}:
 Any["DEBRVs", "DEBRV1", "NOSVG", "NOSVGt"]
 Any["DEBRVs", "DEBRV1", "NOSVG", "NOBGO", "NOBGOt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "RUKGDt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "FIKTK", "FIRAU", "FIRAUt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "FIKTK", "FIKTKt"]
 Any["DEBRVs", "DEBRV3", "DKAAR", "SEGOT3", "SEGOTt"]
 Any["DEBRVs", "DEBRV3", "DKAAR", "SEGOT3", "SEGOT4", "RULED", "RULEDt"]

In [222]:
candidate_paths[reduced_cost .< 0]

7-element Vector{Any}:
 Any["DEBRVs", "DEBRV1", "NOSVG", "NOSVGt"]
 Any["DEBRVs", "DEBRV1", "NOSVG", "NOBGO", "NOBGOt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "RUKGDt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "FIKTK", "FIRAU", "FIRAUt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "FIKTK", "FIKTKt"]
 Any["DEBRVs", "DEBRV3", "DKAAR", "SEGOT3", "SEGOTt"]
 Any["DEBRVs", "DEBRV3", "DKAAR", "SEGOT3", "SEGOT4", "RULED", "RULEDt"]

In [131]:
solution_summary(flowModel)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 3.42021e+06
  Objective bound    : -Inf
  Dual objective value : 3.42021e+06

* Work counters
  Solve time (sec)   : 1.89781e-04


In [132]:
value.(x[P])


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
And data, a 47-element Vector{Float64}:
  20.0
  12.0
   0.0
   7.0
  10.0
   0.0
   0.0
   0.0
 121.0
   0.0
   ⋮
  32.0
 162.0
 276.0
 203.0
   6.0
 298.0
 158.0
   0.0
   0.0

In [133]:
value.(y[S])

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4]
And data, a 4-element Vector{Float64}:
 0.18222222222222223
 1.0
 1.0
 1.0

In [42]:
candidate_paths[reduced_cost .< 0]

8-element Vector{Any}:
 Any["DEBRVs", "DEBRV1", "NOSVG", "NOSVGt"]
 Any["SEGOTs", "SEGOT3", "NOKRS", "DEBRV3", "DEBRVt"]
 Any["DEBRVs", "DEBRV1", "NOSVG", "NOBGO", "NOBGOt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "RUKGDt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "FIKTK", "FIRAU", "FIRAUt"]
 Any["DEBRVs", "DEBRV2", "RUKGD", "FIKTK", "FIKTKt"]
 Any["DEBRVs", "DEBRV3", "DKAAR", "SEGOT3", "SEGOTt"]
 Any["DEBRVs", "DEBRV3", "DKAAR", "SEGOT3", "SEGOT4", "RULED", "RULEDt"]

In [15]:
dual_Y_ij[dual_Y_ij .< 0]

8-element Vector{Float64}:
 -341.4444444444446
 -102.99999999999994
 -553.9999999999998
 -591.0000000000002
 -358.0
 -358.0
 -517.0
 -358.0

In [71]:
flowModel = Model(HiGHS.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, 0 <= y[S] <= 1)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum(current_costs[j]*x[j] for j in P))
#@constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j+n_ff] for j in A) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
#@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
#@constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= baltic_fleet.Quantity[i])
#optimize!(flowModel)

# dual_Y_k = [dual(flowModel[:Y_k][i]) for i in K]
# dual_Y_ij = [dual(flowModel[:Y_ij][i]) for i in ij]
# append!(dual_Y_ij, zeros(Float64, length(edges) - length(ports_adj)))


MethodError: MethodError: no method matching zero(::Type{Any})

Closest candidates are:
  zero(::Type{Union{Missing, T}}) where T
   @ Base missing.jl:105
  zero(!Matched::Union{Type{P}, P}) where P<:Dates.Period
   @ Dates /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/Dates/src/periods.jl:51
  zero(!Matched::GenericAffExpr)
   @ JuMP ~/.julia/packages/JuMP/ToPd2/src/aff_expr.jl:201
  ...


In [30]:
current_commodities[111:end]

14-element Vector{Any}:
 22
  1
  4
  6
  7
  9
 10
 11
 13
 15
 16
 18
 20
 22

In [35]:
i_j

62-element Vector{Any}:
 "DEBRVs|DEBRV1"
 "DEBRV1|DEBRVt"
 "DEBRVs|DEBRV2"
 "DEBRV2|DEBRVt"
 "DEBRVs|DEBRV3"
 "DEBRV3|DEBRVt"
 "DEBRVs|DEBRV4"
 "DEBRV4|DEBRVt"
 "NOSVGs|NOSVG"
 "NOSVG|NOSVGt"
 ⋮
 "FIRAU|PLGDY"
 "PLGDY|DEBRV2"
 "DEBRV3|DKAAR"
 "DKAAR|SEGOT3"
 "SEGOT3|NOKRS"
 "NOKRS|DEBRV3"
 "DEBRV4|SEGOT4"
 "SEGOT4|RULED"
 "RULED|DEBRV4"

Berfore the master problem is formulated, the $c_{y}$ constants are calculated which correspond to the fixed port costs (more might be added later).

Moreover, the sum of all paths must be equal to $d_{k}$, in order to fulfill the demand. However later on, a new expensive penalty edge is added which penalizes all the containers that are not delivered to the commodity port.

As the decision variables, $x_{i,j}$ represent the edges in the liner shipping network (eventually paths), the variables are initialized by pairing the neigbor ports in the services above. Thereby, the master problem is reduced to only respect the strict subset of all feasible patterns i.e. patterns using the generated services.

In [ ]:
# edges with capacity constraint
i_j = []
for edge in edges
    if edge ∈ (penalty_adj ∪ dummy_adj)
        break
    end
    push!(i_j, string(edge[1], "|", edge[2]))
end

# Paths and costs from shortest path
for i in 1:size(baltic_demand, 1)
    path, cost = dijkstra(get_G(edges), string(baltic_demand.Origin[i],"s"), string(baltic_demand.Destination[i],"t"))
    push!(current_paths, path)
    push!(current_costs, cost)
    append!(current_commodity, i)
end

# Initializing parameters
n_k = size(baltic_demand.Origin, 1) # Number of commodities
K = 1:n_k

n_s = size(services_w_transshipment, 1) # Number of services
S = 1:n_s

n_ff = size(penalty_adj, 1) # Number of penalty edges
ff = 1:n_ff
index_ff = 1:n_k # Index of ff paths - Could be hard coded

n_p = size(current_paths, 1) # Number of paths
P = 1:n_p
index_p = n_ff+1:n_p  # Index of non-ff paths - Could be hardcoded

n_ij = size(i_j, 1) # Number of edges
ij = 1:n_ij # Range of edges

n_mv = size(baltic_fleet, 1)
v = 1:n_mv
;

In [ ]:
# Initial paths including shortest path for all commodities and activated penalty edges
aij = Matrix{Float64}(undef, n_ij, 0) # 1 if path p for commodity k uses edge (i,j), else 0
uij = Matrix{Float64}(undef, n_ij, 0) # vessel capacity if service s uses edge (i,j), else 0

for i in index_p
    ak = zeros(Float64, n_ij)
    for j in 2:length(paths[i])
        ak[findall(x -> x == string(paths[i][j-1], "|", paths[i][j]), i_j)[1]] = 1
    end
    aij = [aij ak]
end


for i in 1:length(services_w_transshipment)
    us = zeros(Float64, n_ij)
    for j in 5:length(services_w_transshipment[i])
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
        us[findall(x -> x == string(services_w_transshipment[i][j-1], "|", services_w_transshipment[i][j-1][1:5], "t"), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
        us[findall(x -> x == string(services_w_transshipment[i][j-1][1:5], "s", "|", services_w_transshipment[i][j-1]), i_j)[1]] = fleet_data[fleet_data."Vessel class" .== services_w_transshipment[i][3], 2][1]
    end
    uij = [uij us]
end



In [ ]:
flowModel = Model(HiGHS.Optimizer)
@variable(flowModel, x[P] >= 0)
@variable(flowModel, 0 <= y[S] <= 1)
@objective(flowModel, Min, sum(services_w_transshipment[i][1]*y[i] for i in S) + sum(costs[j]*x[j] for j in P))
#@constraint(flowModel, Y_ij[i=1:n_ij], sum(aij[i, j] * x[j+n_ff] for j in K) <= sum(uij[i, j] * y[j] for j in S)) # Paths that uses edge ij
@constraint(flowModel, Y_k[i=1:n_k], sum(PK[i, j] * x[j] for j in P) == baltic_demand.FFEPerWeek[i]) # Paths for commodity k ∈ K
@constraint(flowModel, M_v[i=1:n_mv], sum(mvs[i, j] * y[j] for j in S) <= baltic_fleet.Quantity[i])
print(flowModel)
optimize!(flowModel)
#solution_summary(flowModel)
;